# Exercício: Gerar Logótipos Sintéticos com DCGAN
* **Descrição:** Treina um DCGAN para criar protótipos de logótipos que ajudem equipas de marketing a testar identidades visuais sem designer humano.
* **Dataset:** Conjunto de imagens de logos
  * **Download:** https://www.kaggle.com/datasets/dimka11/lld-logo-files

## Passo a passo

1. Carregar logos 32×32 em escalas [-1,1]
2. Criar um modelo Gerador simples
3. Criar um modelo Discriminador
4. Ciclo treino simplificado
5. Mostrar o resultado

## Download do Dataset


*   Do Kaggle: https://www.kaggle.com/datasets/dimka11/lld-logo-files
*   Do AWS:

In [ ]:
!wget https://genaiacademy.s3.eu-west-3.amazonaws.com/logo-files/logo-files.zip
!unzip logo-files.zip

In [ ]:
#!pip install tensorflow matplotlib
import tensorflow as tf, matplotlib.pyplot as plt

In [ ]:
# 1. carregar logos 32×32 em escalas [-1,1]
imgs = tf.data.Dataset.list_files('logos/*.png').map(
    lambda f: (tf.image.convert_image_dtype(tf.image.resize(
        tf.io.decode_png(tf.io.read_file(f)), (32,32)), tf.float32)*2)-1)
batch = imgs.shuffle(1000).batch(128)

In [ ]:
# 2. Gerador simples
def gerador():
    modelo = tf.keras.Sequential([
        tf.keras.layers.Dense(8*8*256, use_bias=False, input_shape=(100,)),
        tf.keras.layers.Reshape((8,8,256)),
        tf.keras.layers.Conv2DTranspose(128,4,2,'same',use_bias=False,activation='relu'),
        tf.keras.layers.Conv2DTranspose(3,4,2,'same',use_bias=False,activation='tanh')
    ])
    return modelo

In [ ]:
# 3. Discriminador
def discriminador():
    m = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64,4,2,'same',input_shape=(32,32,3)),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return m

In [ ]:
gen, disc = gerador(), discriminador()
gan = tf.keras.Sequential([gen, disc])
disc.compile('adam','binary_crossentropy')
disc.trainable = False
gan.compile('adam','binary_crossentropy')

# 4. ciclo treino simplificado
for ep in range(30):
    for real in batch:
        z = tf.random.normal((real.shape[0],100))
        fake = gen(z)
        disc.trainable = True
        disc.train_on_batch(tf.concat([real,fake],0),
                            tf.concat([tf.ones(real.shape[0]),tf.zeros(real.shape[0])],0))
        disc.trainable = False
        gan.train_on_batch(z, tf.ones(real.shape[0]))
    print(f'Época {ep+1} concluída')

In [ ]:
# 5. mostrar output
plt.imshow((gen(tf.random.normal((1,100)))[0]+1)/2); plt.axis('off')